In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery, event, trial, imaging, scan, model
from adamacs.ingest import session as isess
from adamacs.helpers import stack_helpers as sh
from adamacs.ingest import behavior as ibe
import datajoint as dj
from rspace_client.eln import eln
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dj.__version__

[2023-05-12 17:28:51,955][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-05-12 17:28:52,001][INFO]: Connected tobiasr@172.26.128.53:3306


'0.14.0'

1. Get latest dataset from Natasha from database
2. load 2p movie to array. 
3. load top-video to array
4. find the Start and end frames of the movie with respect to the tracking gate 

In [33]:
# defining queries

user = "NK"
userquery = (session.SessionUser * subject.User() & f'initials = "{user}"')
# get the latest session
usesession = session.Session.fetch(order_by='session_datetime DESC', as_dict=True)[0]["session_id"]
sessionquery = session.Session &  f'session_id = "{usesession}"'

scansi = (scan.Scan & sessionquery).fetch('scan_id')[0]
scanquery = scan.Scan & f'scan_id = "{scansi}"'


In [18]:
from pathlib import Path
import skvideo.io

# get the main data directory of the latest session of the specified user
# dir = (session.SessionDirectory & sessionquery).fetch('session_dir')
dir = (scan.ScanPath & scanquery).fetch('path')
directory = Path(dir[0])
pattern = '*top_video*.mp4'
files = list(directory.glob(pattern))
videodata = skvideo.io.vread(str(files[0]))


In [41]:
# get the blinking movie data of the latest session of the specified user
directory = Path(dir[0] + "/suite2p/plane0/reg_tif")
pattern = '*80_frame*.mp4'
files = list(directory.glob(pattern))
blinkvideodata = skvideo.io.vread(str(files[0]))

In [42]:
print(videodata.shape)
print(blinkvideodata.shape)

(36157, 1000, 1000, 3)
(9230, 256, 256, 3)


In [76]:
sh.display_volume_z(videodata,1)

interactive(children=(IntSlider(value=0, description='z', max=36156), Output()), _dom_classes=('widget-interac…

In [75]:
sh.display_volume_z(blinkvideodata,1)

interactive(children=(IntSlider(value=0, description='z', max=9229), Output()), _dom_classes=('widget-interact…

In [48]:
event.EventType()

event_type,event_type_description
arena_LED,
aux_bpod_cam,
aux_bpod_reward,
aux_bpod_tone,
aux_bpod_visual,
aux_cam,
HARP_gate,
main_track_gate,
mini2p_frames,
mini2p_lines,


In [55]:
cameratimestamps = (event.Event()  &  "event_type='aux_cam'" &  scanquery ).fetch('event_start_time')
twoptimestamps = (event.Event()  &  "event_type='mini2p_frames'" &  scanquery ).fetch('event_start_time')
cameratimestamps = cameratimestamps - twoptimestamps[0]

In [70]:
sync_video1, sync_video2, frame_rate_video1, frame_rate_video2 = synchronize_videos(blinkvideodata, videodata, twoptimestamps, cameratimestamps)



In [77]:
video1 = blinkvideodata
video2 = videodata
timestamps_video1 = twoptimestamps
timestamps_video2 = cameratimestamps

In [72]:
# Print the shapes and frame rates
print("Synchronized Video 1 shape:", sync_video1.shape)
print("Synchronized Video 2 shape:", sync_video2.shape)
print("Frame rate of Video 1:", frame_rate_video1)
print("Frame rate of Video 2:", frame_rate_video2)

Synchronized Video 1 shape: (0, 256, 256, 3)
Synchronized Video 2 shape: (36157, 1000, 1000, 3)
Frame rate of Video 1: 1819.0418005163478
Frame rate of Video 2: 7125.795707613173


In [78]:
import numpy as np

# def synchronize_and_truncate_videos(video1, video2, timestamps_video1, timestamps_video2):
# Calculate time difference between the two videos
time_diff = timestamps_video1[0] - timestamps_video2[0]

# # Calculate frame rates based on the time difference and frame count
#     interframe_intervals = np.diff(timestamps)

#     # Calculate the average interframe interval
#     average_interval = np.mean(interframe_intervals)

#     # Calculate the frame rate as the reciprocal of the average interframe interval
#     frame_rate = 1.0 / average_interval

frame_rate_video1 = 1/np.median(np.diff(timestamps_video1))
frame_rate_video2 = 1/np.median(np.diff(timestamps_video2))

# Determine the starting frame indices for each video
start_frame_video1 = int(time_diff * frame_rate_video1)
start_frame_video2 = 0  # Assuming video 2 starts at the beginning

# Calculate the length of synchronized arrays
sync_len_video1 = video1.shape[0] - start_frame_video1
sync_len_video2 = video2.shape[0] - start_frame_video2

# Create synchronized arrays
sync_video1 = np.zeros((sync_len_video1, *video1.shape[1:]), dtype=video1.dtype)
sync_video2 = np.zeros((sync_len_video2, *video2.shape[1:]), dtype=video2.dtype)

# Extract synchronized frames from video 1
sync_video1[:] = video1[start_frame_video1:]

# Extract synchronized frames from video 2
sync_video2[:] = video2[start_frame_video2:]

# Calculate the end time of video1
end_time_video1 = timestamps_video1[-1]

# Find the corresponding frame index in video2 based on the end time of video1
end_frame_video2 = int(end_time_video1 * frame_rate_video2)

# Truncate video2 to match the end time of video1
sync_video2 = sync_video2[:end_frame_video2]

return sync_video1, sync_video2, frame_rate_video1, frame_rate_video2


SyntaxError: 'return' outside function (735940306.py, line 38)

In [91]:
int(time_diff * frame_rate_video2)

298

In [89]:
frame_rate_video1

15.384615384615923

In [83]:
timestamps_video1[0] - timestamps_video2[0]

5.0741

In [56]:
model.VideoRecording()

session_id,scanner,recording_id,recording_start_time
